### 1. INTRODUCTION

**The project was aimed to classify crop types using time-series Sentinel-2 images with Random Forest Algorithm in Google Earth Engine (GEE) with Python API. The study region is part of Van Wert, Ohio USA.**

### 2. IMPORT LIBRARIES AND DEFINE FUNCTIONS FOR VISUALIZATION

In [1]:
# Import libraries
import ee
import geemap
import geedim as gd
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler

/home/maxi/docs/beca/Crop-Classification/.venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
# Trigger the GEE authentication flow
ee.Authenticate()

True

In [3]:
# Initialize the library
ee.Initialize()

In [4]:
# Initializes the Google Earth Engine Python API
geemap.ee_initialize()

## Define Functions for Visualizations

In [5]:
# Function to visualize single vector layer
def Map(layer, viz_params, name):
    """
    """
    
    map = geemap.Map(center=[40.7959, -84.4795], zoom = 12)
    map.add_basemap("SATELLITE")

    styled_layer = layer.style(**viz_params)
    map.addLayer(styled_layer, {}, name)
    return map

In [6]:
# Function to visualize single raster layer
def rasterMap(layer, viz_params, name):
    """
    """
    
    map = geemap.Map(center=[40.7959, -84.4795], zoom = 12)
    map.add_basemap("SATELLITE")

    map.addLayer(layer, viz_params, name)
    return map

In [7]:
# Function to visualize two (2) layers using slider Map

def sliderMap(layer1params, layer2params):
    """

    """

    ee_object1, vis_params1, name1 = layer1params
    ee_object2, vis_params2, name2 = layer2params
    
    left_layer = geemap.ee_tile_layer(ee_object1, vis_params1, name=name1)
    right_layer = geemap.ee_tile_layer(ee_object2, vis_params2, name=name2)
    
    map = geemap.Map(center=(40.7959, -84.4795), zoom =12)

    map.split_map(left_layer, right_layer)
    return map

In [8]:
# Function to visualize four (4) layers using a linked map
def linkedMap(layers, viz_params, labels):
    """
    """

    map = geemap.linked_maps(
        rows=2,
        cols=2,
        height="450px",
        center= [40.7959, -84.4795],
        zoom=11,
        ee_objects=layers,
        vis_params= viz_params,
        labels= labels,
        label_position="topright"
    )
    return map

### 3. Data Downloading and Processing

#### Boundary Data

In [9]:
# Define the study region extent
# The study area is part of Van Wert, Ohio USA.
roi = ee.Geometry.Polygon(
    [[-84.70947870462608,40.64732007950524],
    [-84.20342096536827,40.64732007950524],
    [-84.20342096536827,40.89536512997029],
    [-84.70947870462608,40.89536512997029],
    [-84.70947870462608,40.64732007950524]])

In [10]:
# Visualize the study region/roi
Map = geemap.Map(center=[40.7959, -84.4795], zoom=7)  

# Convert the roi from feature collection to a GeodataFrame
roi_layer = geemap.ee_to_gdf(ee.FeatureCollection(roi))
Map.add_gdf(roi_layer, style={"fillColor": "rgba(0, 0, 0, 0)", "color": "red", "weight": 2})

Map.add_basemap("SATELLITE")
Map

Map(center=[40.7959, -84.4795], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

####  NAIP: National Agriculture Imagery Program 

In [11]:
# Load the National Agriculture Imagery Program (NAIP) satellite imagery (0.6m)
naip = (ee.ImageCollection("USDA/NAIP/DOQQ")
        .filter(ee.Filter.date("2021-06-20", "2021-08-30"))
        .filterBounds(roi))

naip = naip.median()

In [12]:
# Visualize the naip layer
viz_params_naip = {
    "min": 0,
    "max": 255,
    "gamma": 0.9
}

rasterMap(naip.select(["R", "G", "B"]), viz_params_naip, "NAIP")

Map(center=[40.7959, -84.4795], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

#### ESA Sentinel-2 Time-Series Images

In [13]:
## Define a function that download and process time-series sentinel-2 images
def s2Process(dates, extent, cloud):

    """

    """
    start_date, end_date = dates
    
    # Filter and download Sentinel-2 image collection 
    def imgDownload():
        
        img_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                    .filterDate(start_date, end_date)
                    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud))
                    .filterBounds(extent))
        print(f"There are {img_collection.size().getInfo()} Sentinel-2 Images")

        s2_default_proj = ee.Image(img_collection.first()).select("B4").projection()
        return img_collection

    # Function for cloud masking
    def s2ClearSky(image):
      
        scl = image.select('SCL')
        clear_sky_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(11))
        return image.updateMask(clear_sky_pixels)
    
    # Function to apply scale factor to convert pixel values to reflectance
    def scaleBands(image):
        return image.multiply(0.0001).copyProperties(image, ['system:time_start'])
    
    # Function for resampling 20 meter bands to 10m
    def bandResample(image):
       
        bands_10m = image.select(["B2", "B3", "B4", "B8"])
        bands_20m = image.select(["B5","B6", "B7", "B8A", "B11", "B12"]).resample("bilinear").reproject(crs=bands_10m.projection(), scale=10)
        return bands_10m.addBands(bands_20m)


    # Function to compute spectral indices
    def indices(image):
        
        ndvi = image.normalizedDifference(["B8", "B4"]).rename("ndvi")
        gndvi = image.normalizedDifference(["B8", "B3"]).rename("gndvi")
        mndwi = image.normalizedDifference(["B3", "B11"]).rename("mndwi")
        
        return image.addBands([ndvi, mndwi, gndvi])
    
    # Donwload the image collection
    s2_collection = imgDownload()

    # Extract the projection before any processing
    s2_default_proj = ee.Image(s2_collection.first()).select("B4").projection()

    # Apply the functions to remove clouds, resmaple bands, and compute spectral indices
    s2_collection_resampled = (s2_collection
                           .map(s2ClearSky)
                           .map(bandResample)
                           .map(scaleBands)
                           .map(indices))
                           #.select(["B2", "B3", "B4", "B8", "B11", "B12", "ndvi"]))
    
    # List of months with their end dates
    months = [
        {'name': 'May', 'end': 31},
        {'name': 'June', 'end': 30},
        {'name': 'July', 'end': 31},
        {'name': 'August', 'end': 31},
        {'name': 'September', 'end': 30},
        {'name': 'October', 'end': 31}
    ]

    # Function to create monthly composites
    def MonthlyComposite(month, index):
        start = ee.Date.fromYMD(2023, index + 5, 1)
        end = ee.Date.fromYMD(2023, index + 5, month['end'])

        monthly_images = s2_collection_resampled.filterDate(start, end)
        
        count = monthly_images.size().getInfo()
        print(f"Month: {month['name']}, Image Count: {count}")  
        if count == 0:
            print(f"No images found for month: {month['name']}")
            return ee.Image().set('month', month['name'])  

        composite = (monthly_images.median()
                      .clip(extent)
                      .set('month', month['name'])
                      .setDefaultProjection(s2_default_proj))

        return composite

    # Create monthly composites for May to October
    monthly_composites = [MonthlyComposite(month, index) for index, month in enumerate(months)]
    

    # Stack the monthly composites
    stacked_composite = ee.Image.cat(monthly_composites)

    # View the properties of the stacked composite
    print('Monthly Composites:', stacked_composite.getInfo())

    # Check the bands in the stacked composite
    stacked_band_names = stacked_composite.bandNames().getInfo()
    print(f"The following are the bands in the stacked composite: {stacked_band_names}")

    # Check the CRS of the stacked composite
    stacked_proj = stacked_composite.select("B4").projection().crs().getInfo()
    print(f"The CRS of the stacke composite is: {stacked_proj}")

    return stacked_composite


In [14]:
# Load and process Sentinel-2 Images using the function
dates = ("2023-04-01", "2023-10-31")

s2_monthly_median = s2Process(dates, roi, 25)

There are 62 Sentinel-2 Images
Month: May, Image Count: 15
Month: June, Image Count: 5
Month: July, Image Count: 5
Month: August, Image Count: 9
Month: September, Image Count: 7
Month: October, Image Count: 12
Monthly Composites: {'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 6.5535000000000005}, 'dimensions': [25869, 22930], 'origin': [-1719, -5751], 'crs': 'EPSG:32616', 'crs_transform': [10, 0, 600000, 0, -10, 4600020]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 6.5535000000000005}, 'dimensions': [25869, 22930], 'origin': [-1719, -5751], 'crs': 'EPSG:32616', 'crs_transform': [10, 0, 600000, 0, -10, 4600020]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 6.5535000000000005}, 'dimensions': [25869, 22930], 'origin': [-1719, -5751], 'crs': 'EPSG:32616', 'crs_transform': [10, 0, 600000, 0, -10, 4600020]}, {'id': 'B8', 'data_type': {'

In [15]:
# A large percentage of pixels was masked of the July Image Composite because of clouds
s2_monthly_median = s2_monthly_median.select(
    ['B2', 'B3', 'B4', 'B8', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12', 'ndvi', 'mndwi', 'gndvi', 'B2_1', 'B3_1', 'B4_1', 'B8_1', 'B5_1', 'B6_1', 'B7_1', 'B8A_1', 'B11_1', 'B12_1', 'ndvi_1', 'mndwi_1', 'gndvi_1', 'B2_3', 'B3_3', 'B4_3', 'B8_3', 'B5_3', 'B6_3', 'B7_3', 'B8A_3', 'B11_3', 'B12_3', 'ndvi_3', 'mndwi_3', 'gndvi_3', 
    'B2_4', 'B3_4', 'B4_4', 'B8_4', 'B5_4', 'B6_4', 'B7_4', 'B8A_4', 'B11_4', 'B12_4', 'ndvi_4', 'mndwi_4', 'gndvi_4', 'B2_5', 'B3_5', 'B4_5', 'B8_5', 'B5_5', 'B6_5', 'B7_5', 'B8A_5', 'B11_5', 'B12_5', 'ndvi_5', 'mndwi_5', 'gndvi_5']
)

In [16]:
# Visualize the Sentinel-2 Median composite image in true colour

viz_params_s2_rgb = {
    "min": 0.0,
    "max": 0.3,
    "bands": ["B4_4", "B3_4", "B2_4"],
    "gamma" : 0.9
}

rasterMap(s2_monthly_median, viz_params_s2_rgb, "Sentinel-2 Median Composite")

Map(center=[40.7959, -84.4795], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

### 4. Training and Testing Data Extraction

**Training and Testing Samples were extracted from an existing Land cover Data (CDL) of 30m Resolution**

In [17]:
# Load the existing USDA NASS Cropland Data Layers (CDL) of 30m resolution
cdl = (ee.ImageCollection("USDA/NASS/CDL")
                  .filter(ee.Filter.date("2023-01-01", "2023-12-31"))
                  .filterBounds(roi)
                  .select("cropland")
                  .first())

cdl_crop = cdl.clip(roi)

In [18]:
# Visualize the CDL layer
viz_params_cdl = {}

rasterMap(cdl_crop, viz_params_cdl, "Cropland Data Layer (30m)")

Map(center=[40.7959, -84.4795], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

In [19]:
# Check the bands in the CDL (30m)
cdl_crop.bandNames()

In [20]:
# Get the properties of the CDL (30m)
cdl_info = cdl_crop.toDictionary().getInfo()
print(cdl_info.keys())

dict_keys(['cropland_class_names', 'cropland_class_palette', 'cropland_class_values', 'cultivated_class_names', 'cultivated_class_palette', 'cultivated_class_values'])


In [21]:
# Get the number of the unique crop types/landcover classes
len(cdl_info["cropland_class_values"])

456

In [38]:
# Select just the 'cropland_class_names', 'cropland_class_palette', and 'cropland_class_values' keys
cdl_name = cdl_info["cropland_class_names"]
cdl_value = cdl_info["cropland_class_values"]
cdl_palette = cdl_info["cropland_class_palette"]

# Hacemos split y convertimos a integer
names_list = cdl_name.split(',')
values_list = [int(x) for x in cdl_value.split(',')]

cdl_name_value = dict(zip(names_list, values_list))
print(cdl_name_value)

{'Background': 0, 'Corn': 1, 'Cotton': 2, 'Rice': 3, 'Sorghum': 4, 'Soybeans': 5, 'Sunflower': 6, 'Peanuts': 10, 'Tobacco': 11, 'Sweet Corn': 12, 'Pop or Orn Corn': 13, 'Mint': 14, 'Barley': 21, 'Durum Wheat': 22, 'Spring Wheat': 23, 'Winter Wheat': 24, 'Other Small Grains': 25, 'Dbl Crop WinWht/Soybeans': 26, 'Rye': 27, 'Oats': 28, 'Millet': 29, 'Speltz': 30, 'Canola': 31, 'Flaxseed': 32, 'Safflower': 33, 'Rape Seed': 34, 'Mustard': 35, 'Alfalfa': 36, 'Other Hay/Non Alfalfa': 37, 'Camelina': 38, 'Buckwheat': 39, 'Sugarbeets': 41, 'Dry Beans': 42, 'Potatoes': 43, 'Other Crops': 44, 'Sugarcane': 45, 'Sweet Potatoes': 46, 'Misc Vegs & Fruits': 47, 'Watermelons': 48, 'Onions': 49, 'Cucumbers': 50, 'Chick Peas': 51, 'Lentils': 52, 'Peas': 53, 'Tomatoes': 54, 'Caneberries': 55, 'Hops': 56, 'Herbs': 57, 'Clover/Wildflowers': 58, 'Sod/Grass Seed': 59, 'Switchgrass': 60, 'Fallow/Idle Cropland': 61, 'Forest': 63, 'Shrubland': 152, 'Barren': 131, 'Cherries': 66, 'Peaches': 67, 'Apples': 68, 'Gra

In [39]:
# Get unique crop class values and the number of pixels for each class from the CDL (30m)
crop_class_pixel_roi = cdl_crop.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=roi,
    scale=30,
    maxPixels=1e7,  
    bestEffort=True 
).getInfo()

print(crop_class_pixel_roi)

{'cropland': {'1': 377571.7019607844, '111': 9083.658823529411, '12': 43, '121': 49617.50588235295, '122': 42253.03921568626, '123': 17837.7568627451, '124': 5258.474509803922, '13': 19800.647058823528, '131': 929.4196078431372, '141': 76396.20392156864, '142': 79.8078431372549, '143': 138, '152': 21, '176': 45280.141176470584, '190': 9393.137254901962, '195': 3004.901960784314, '205': 7, '21': 1, '225': 12, '229': 10, '236': 5, '24': 53332.607843137266, '254': 2, '26': 3442.2666666666664, '27': 380, '28': 163.04313725490195, '29': 24, '36': 10039.439215686274, '37': 6494.019607843137, '39': 1, '4': 588, '42': 6, '43': 6, '5': 576024.0705882349, '54': 150, '58': 180, '59': 8, '6': 4, '61': 1, '68': 4, '70': 2}}


In [40]:
# Check the CRS of the cdl_crop 
cdl_crop_proj = cdl_crop.select(0).projection()
cdl_crop_crs = cdl_crop_proj.crs().getInfo()
print(f"The CRS of the Crop Data Layer  is: {cdl_crop_crs}")

# Check the CRS of the Sentinel-2 Monthly Median Composite
s2_median_proj = s2_monthly_median.select(0).projection()
s2_median_crs = s2_median_proj.crs().getInfo()
print(f"The CRS of the Sentinel-2 monthly median composite is:{s2_median_crs}")

The CRS of the Crop Data Layer  is: EPSG:5070
The CRS of the Sentinel-2 monthly median composite is:EPSG:32616


In [41]:
# Define grid size and projection
grid_scale = 10
grid_proj = ee.Projection("EPSG:5070").atScale(grid_scale)

In [42]:
# Reproject the Sentinel-2 Monthly Median Composite
s2_monthly_median = s2_monthly_median.reproject(
                    crs = grid_proj
                    )

# Re-check the CRS of the Sentinel-2 Monthly Median Composite
s2_median_proj = s2_monthly_median.select(0).projection()
s2_median_crs = s2_median_proj.crs().getInfo()
print(f"The CRS of the Reprojected S-2 monthly median composite is:{s2_median_crs}")

The CRS of the Reprojected S-2 monthly median composite is:EPSG:5070


In [43]:
# Aggregate the cdl_crop to the same grid (10m) as the s2_composite and take the values of the highest occuring class
cdl_crop_resampled = cdl_crop.reduceResolution(
    reducer = ee.Reducer.mode(),
    maxPixels = 1024
).reproject(
    crs = grid_proj
)

In [44]:
# Check the spatial resolution of the resampled cdl_crop
cdl_crop_resampled_scale = cdl_crop_resampled.projection().nominalScale().getInfo()
print(f"The spatial resolution of the resampled CDL is: {cdl_crop_resampled_scale} meters")

The spatial resolution of the resampled CDL is: 10 meters


In [45]:
# Visualize the resampled CDL (10m)
rasterMap(cdl_crop_resampled, viz_params_cdl, "Cropland Data Layers (Resampled)")

Map(center=[40.7959, -84.4795], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

In [46]:
# Get unique crop class values and the number of pixels for each class from the CDL (30m)
crop_class_pixel_roi = cdl_crop_resampled.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=roi,
    scale=10,
    maxPixels=1e7,  
    bestEffort=True 
).getInfo()

print(crop_class_pixel_roi)

{'cropland': {'1': 2890144.8078431394, '111': 69556.36078431373, '12': 322, '121': 376330.3960784312, '122': 320583.2705882353, '123': 133318.59215686272, '124': 39601.72941176471, '13': 151527.9568627451, '131': 6896.274509803921, '141': 586992.3647058824, '142': 572.1803921568628, '143': 970, '152': 132, '176': 343394.7019607843, '190': 70051.68627450979, '195': 22477.309803921573, '205': 52, '21': 8, '225': 81, '229': 72, '236': 40, '24': 408562.0823529409, '254': 16, '26': 26189.75294117647, '27': 2907.6235294117646, '28': 1209.792156862745, '29': 183, '36': 75791.12156862745, '37': 47881.549019607846, '39': 6, '4': 4507, '42': 52, '43': 38, '5': 4411901.6627450995, '54': 1139, '58': 1292, '59': 58, '6': 30, '61': 5, '68': 33, '70': 12}}


In [47]:
# Select the classes with the considerable number of pixels (majority classes) from the Resampled CDL
unique_crop_class_roi = []

for label, pixels in crop_class_pixel_roi["cropland"].items():
    if pixels >  26189:
    #if pixels >   47881:
        unique_crop_class_roi.append(label)
        unique_crop_class_roi = list(map(int, unique_crop_class_roi))

# I added this class separately
unique_crop_class_roi.append(195)
print(np.sort(unique_crop_class_roi))

[  1   5  13  24  26  36  37 111 121 122 123 124 141 176 190 195]


In [48]:
# Select grids for the following major crop classes/ Filter major crop class
cdl_crop_masked = (cdl_crop_resampled.eq(1)
                 .Or(cdl_crop_resampled.eq(5))
                 .Or(cdl_crop_resampled.eq(13))
                 .Or(cdl_crop_resampled.eq(24))
                 .Or(cdl_crop_resampled.eq(26))
                 .Or(cdl_crop_resampled.eq(36))
                 .Or(cdl_crop_resampled.eq(37))
                 .Or(cdl_crop_resampled.eq(111))
                 .Or(cdl_crop_resampled.eq(121))
                 .Or(cdl_crop_resampled.eq(122))
                 .Or(cdl_crop_resampled.eq(123))
                 .Or(cdl_crop_resampled.eq(124))
                 .Or(cdl_crop_resampled.eq(141))
                 .Or(cdl_crop_resampled.eq(176))
                 .Or(cdl_crop_resampled.eq(190))
                 .Or(cdl_crop_resampled.eq(195))
                 )

In [49]:
# Create a mask of the pixels with the major landcover/crop type alone
cdl_crop_resampled_masked = cdl_crop_resampled.updateMask(cdl_crop_masked)

In [50]:
# Get the unique values in the masked CDL
reclass_cdl_unique_values = cdl_crop_resampled_masked.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=roi,
    scale=10,
    maxPixels=1e10
).get("cropland").getInfo()

# Print the unique values
print(reclass_cdl_unique_values.keys())

dict_keys(['1', '111', '121', '122', '123', '124', '13', '141', '176', '190', '195', '24', '26', '36', '37', '5'])


In [51]:
# Visualize the masked CDL
viz_params_cdl_roi = {"min": 1, "max":16, "palette" : ["#ffd300", "#4970a3", "#dda50a", "#707000",  "#267000", "#a57000",  "#ffa5e2", "#a5f28c",  "#999999",  "#999999", "#999999", "#93cc93", "#e8ffbf", "#7cafaf", "#7cafaf", "#999999",]}

rasterMap(cdl_crop_resampled_masked, viz_params_cdl_roi, "Cropland Data Layer - MASKED")

Map(center=[40.7959, -84.4795], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

In [52]:
# Reduce the number of classess by combining the "Developed" classes into a single class 
# ['Developed/Open Space', 'Developed/Low Intensity', 'Developed/Med Intensity', 'Developed/High Intensity',]
# [121, 122, 123, 124]

# Define the new value to combine all the old values
to_value = 121  

# Get the cropland band from the resampled masked CDL (10m)
cropland_band = cdl_crop_resampled_masked.select("cropland")

# Remap the old values to the new value (121)
remapped_cropland = (cropland_band
    .where(cropland_band.eq(122), to_value)
    .where(cropland_band.eq(123), to_value)
    .where(cropland_band.eq(124), to_value))

# Replace the original cropland band with the remapped one
cdl_crop_resampled_masked_combined = cdl_crop_resampled_masked.addBands(remapped_cropland.rename("cropland_combined"))

# Remove the old 'cropland' band and add the new 'cropland_combined' band as 'cropland'
cdl_crop_resampled_masked_combined = cdl_crop_resampled_masked_combined.select([band for band in cdl_crop_resampled_masked_combined.bandNames().getInfo() if band != "cropland"]).addBands(remapped_cropland.rename("cropland"))

# Verify the change remapped cropland band
reclass_cdl_unique_values = cdl_crop_resampled_masked_combined.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=roi,  
    scale=grid_scale,
    maxPixels=1e10
).get("cropland").getInfo()

# Print the unique values
print("Unique values after remapping:", reclass_cdl_unique_values.keys())


Unique values after remapping: dict_keys(['1', '111', '121', '13', '141', '176', '190', '195', '24', '26', '36', '37', '5'])


In [53]:
# # Get unique crop class values and the number of pixels for each class from the Combined Masked CDL (10)
unique_values_combined = cdl_crop_resampled_masked_combined.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=roi,
    scale=grid_scale,
    maxPixels=1e9
)

# Get the histogram dictionary
combined_histogram_dict = unique_values_combined.get("cropland").getInfo()

# Print the unique values and their counts
print("Unique values for the new classes in the combined Masked CDL", combined_histogram_dict)


Unique values for the new classes in the combined Masked CDL {'1': 3403104.6980392155, '111': 81859.83137254903, '121': 1025138.7019607842, '13': 178459.98823529415, '141': 690899.1333333334, '176': 404609.72941176477, '190': 82442.81568627452, '195': 26367.12156862745, '24': 480784.8196078431, '26': 30668.749019607843, '36': 89168.00392156863, '37': 56353.149019607845, '5': 5194350.941176471}


In [54]:
# Extract the crop types/land cover classes from the Combined Masked CDL
unique_crop_class_combined = []

for label, pixels in combined_histogram_dict.items():
        unique_crop_class_combined.append(label)
        unique_crop_class_combined = list(map(int, unique_crop_class_combined))
print(np.sort(unique_crop_class_combined))

[  1   5  13  24  26  36  37 111 121 141 176 190 195]


In [55]:
# Add "names" to the selected crop classes(values) 
cdl_name_class_combined = {}

for name, value in cdl_name_value.items():
     if value in unique_crop_class_combined:
        cdl_name_class_combined[name] = value

print(cdl_name_class_combined)

{'Corn': 1, 'Soybeans': 5, 'Pop or Orn Corn': 13, 'Winter Wheat': 24, 'Dbl Crop WinWht/Soybeans': 26, 'Alfalfa': 36, 'Other Hay/Non Alfalfa': 37, 'Open Water': 111, 'Developed/Open Space': 121, 'Deciduous Forest': 141, 'Grass/Pasture': 176, 'Woody Wetlands': 190, 'Herbaceous Wetlands': 195}


In [57]:
# Extract point samples from the the Combined Masked CDL using the stratified random sampling method

num_points = 60  
sampled_points = cdl_crop_resampled_masked_combined.stratifiedSample(
    numPoints= num_points,
    classBand= "cropland", 
    region= roi,
    scale= grid_scale,
    geometries= True
)

# Convert sampled points to a feature collection
sampled_points_fc = ee.FeatureCollection(sampled_points)

# Print the sampled points
print(sampled_points_fc.first().getInfo())

# Check the class balance
def countClasses(feature_collection, class_property):
    class_count = feature_collection.aggregate_histogram(class_property).getInfo()
    return class_count

# Count the number of points in each class
class_balance = countClasses(sampled_points_fc, "cropland")
print("Class Balance:", class_balance)


{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-84.57771735149556, 40.69557290061727]}, 'id': '0', 'properties': {'cropland': 1, 'cropland_combined': 1}}
Class Balance: {'1': 60, '111': 60, '121': 60, '13': 60, '141': 60, '176': 60, '190': 60, '195': 60, '24': 60, '26': 60, '36': 60, '37': 60, '5': 60}


**For some reasons, the error matrix was not working with the default labels for the crop types from the "combined_samples" feature collection. So, I had to convert the collection to a GDF and renamed them sequentially.**

In [59]:
# Convert the point samples to a GeodataFrame
samples_gdf = geemap.ee_to_gdf(sampled_points_fc)

samples_gdf.info()
print(samples_gdf.head())

# Get the unique labels for the crop types
unique_values = samples_gdf["cropland"].unique()

# Convert the existing unique values to a sequence of integers from 1 to n
value_mapping = {value: idx + 1 for idx, value in enumerate(unique_values)}
samples_gdf["cropland"] = samples_gdf["cropland"].map(value_mapping)


print(f"The new classes are: {samples_gdf.cropland.unique()}")


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   geometry           780 non-null    geometry
 1   cropland           780 non-null    int64   
 2   cropland_combined  780 non-null    int64   
dtypes: geometry(1), int64(2)
memory usage: 18.4 KB
                     geometry  cropland  cropland_combined
0  POINT (-84.57772 40.69557)         1                  1
1  POINT (-84.35341 40.78827)         1                  1
2  POINT (-84.66008 40.73437)         1                  1
3   POINT (-84.20672 40.7359)         1                  1
4  POINT (-84.32629 40.66603)         1                  1
The new classes are: [ 1  2  3  4  5  6  7  8  9 10 11 12 13]


In [60]:
# Convert the samples back to a FeatureCollection
sample_ee = geemap.gdf_to_ee(samples_gdf)
sample_ee.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [-84.57771735149556, 40.69557290061727]},
 'id': '0',
 'properties': {'cropland': 1, 'cropland_combined': 1}}

### 5. ML Training and Validation - Random Forest

In [61]:
# Split the samples randomly to training and testing (70, 30)
sample = sample_ee.randomColumn()
training_sample = sample.filter(ee.Filter.lt("random", 0.7))
testing_sample = sample.filter(ee.Filter.gte("random", 0.7))

# Print the size of training and testing samples
print(f"The training sample size: {training_sample.size().getInfo()}")
print(f"The testing sample size: {testing_sample.size().getInfo()}")

print(f"The training sample Example: {training_sample.first().getInfo()}")
print(f"The testing sample Example: {testing_sample.first().getInfo()}")

# Extract unique "classes" values from the training and testing sample
training_ids = training_sample.aggregate_array("cropland").distinct()
training_ids_list = training_ids.getInfo()

# Extract unique "classes" values from the testing set
testing_ids = testing_sample.aggregate_array("cropland").distinct()
testing_ids_list = testing_ids.getInfo()

print("Unique Classes in Training Set:", training_ids_list)
print("Unique Classes in Testing Set:", testing_ids_list)


The training sample size: 548
The testing sample size: 232
The training sample Example: {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-84.57771735149556, 40.69557290061727]}, 'id': '0', 'properties': {'cropland': 1, 'cropland_combined': 1, 'random': 0.5913281387568771}}
The testing sample Example: {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-84.32629429380505, 40.666028957190804]}, 'id': '4', 'properties': {'cropland': 1, 'cropland_combined': 1, 'random': 0.7391851999552607}}
Unique Classes in Training Set: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Unique Classes in Testing Set: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [62]:
# Visualize the training and testing sets without the spectral properties of the image
Map = geemap.Map(center=[40.7959, -84.4795], zoom=10)

# Style dictionaries for training and testing samples
training_style = {"color": "blue", "pointSize": 3}
testing_style = {"color": "red", "pointSize": 3}

# Add training samples to the map with blue color
training_layer = training_sample.style(**training_style)
Map.addLayer(training_layer, {}, "Training Samples")

# Add testing samples to the map with red color
testing_layer = testing_sample.style(**testing_style)
Map.addLayer(testing_layer, {}, "Testing Samples")

# Display the map
Map

Map(center=[40.7959, -84.4795], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

In [63]:
# Print unique classes in the training set
unique_classes_training = training_sample.aggregate_histogram("cropland").getInfo()
print("Unique classes in training set:", unique_classes_training)

# Print unique classes in the testing set
unique_classes_testing = testing_sample.aggregate_histogram("cropland").getInfo()
print("Unique classes in testing set:", unique_classes_testing)

Unique classes in training set: {'1': 44, '10': 40, '11': 42, '12': 43, '13': 42, '2': 40, '3': 49, '4': 37, '5': 43, '6': 43, '7': 41, '8': 40, '9': 44}
Unique classes in testing set: {'1': 16, '10': 20, '11': 18, '12': 17, '13': 18, '2': 20, '3': 11, '4': 23, '5': 17, '6': 17, '7': 19, '8': 20, '9': 16}


In [64]:
# Create a mask of the pixels with the major landcover/crop type alone in the s2_monthly_median
s2_monthly_median_masked = s2_monthly_median.updateMask(cdl_crop_masked)
s2_monthly_median_masked.bandNames().getInfo()

['B2',
 'B3',
 'B4',
 'B8',
 'B5',
 'B6',
 'B7',
 'B8A',
 'B11',
 'B12',
 'ndvi',
 'mndwi',
 'gndvi',
 'B2_1',
 'B3_1',
 'B4_1',
 'B8_1',
 'B5_1',
 'B6_1',
 'B7_1',
 'B8A_1',
 'B11_1',
 'B12_1',
 'ndvi_1',
 'mndwi_1',
 'gndvi_1',
 'B2_3',
 'B3_3',
 'B4_3',
 'B8_3',
 'B5_3',
 'B6_3',
 'B7_3',
 'B8A_3',
 'B11_3',
 'B12_3',
 'ndvi_3',
 'mndwi_3',
 'gndvi_3',
 'B2_4',
 'B3_4',
 'B4_4',
 'B8_4',
 'B5_4',
 'B6_4',
 'B7_4',
 'B8A_4',
 'B11_4',
 'B12_4',
 'ndvi_4',
 'mndwi_4',
 'gndvi_4',
 'B2_5',
 'B3_5',
 'B4_5',
 'B8_5',
 'B5_5',
 'B6_5',
 'B7_5',
 'B8A_5',
 'B11_5',
 'B12_5',
 'ndvi_5',
 'mndwi_5',
 'gndvi_5']

In [65]:
# Visualize the MASKED Sentinel-2 Median composite image in true colour

rasterMap(s2_monthly_median_masked, viz_params_s2_rgb, "Sentinel-2 Monthly Median Composite (MASKED)")

Map(center=[40.7959, -84.4795], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

In [66]:
# Extract the (spectral) properties of the bands in the s2_monthly_median layer to the training points
training_set = s2_monthly_median_masked.sampleRegions(
        collection= training_sample,
        properties = ["cropland"],
        scale= grid_scale
    )

# Inspect the result
training_set.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B11': 0.4077,
  'B11_1': 0.4833,
  'B11_3': 0.17800000000000002,
  'B11_4': 0.1988,
  'B11_5': 0.25105,
  'B12': 0.3407,
  'B12_1': 0.433,
  'B12_3': 0.0772,
  'B12_4': 0.0951,
  'B12_5': 0.17495,
  'B2': 0.10045000000000001,
  'B2_1': 0.1452,
  'B2_3': 0.0415,
  'B2_4': 0.04,
  'B2_5': 0.04345,
  'B3': 0.1338,
  'B3_1': 0.191,
  'B3_3': 0.0616,
  'B3_4': 0.07,
  'B3_5': 0.0635,
  'B4': 0.1778,
  'B4_1': 0.2546,
  'B4_3': 0.0376,
  'B4_4': 0.042,
  'B4_5': 0.09005,
  'B5': 0.2218,
  'B5_1': 0.2851,
  'B5_3': 0.07730000000000001,
  'B5_4': 0.0883,
  'B5_5': 0.119,
  'B6': 0.235,
  'B6_1': 0.29350000000000004,
  'B6_3': 0.31980000000000003,
  'B6_4': 0.3048,
  'B6_5': 0.13965,
  'B7': 0.25525000000000003,
  'B7_1': 0.31120000000000003,
  'B7_3': 0.4666,
  'B7_4': 0.42560000000000003,
  'B7_5': 0.15945,
  'B8': 0.2804,
  'B8A': 0.29005000000000003,
  'B8A_1': 0.3387,
  'B8A_3': 0.48810000000000003,
  'B8A_4': 0.45530000

In [67]:
cdl_name_class_combined

{'Corn': 1,
 'Soybeans': 5,
 'Pop or Orn Corn': 13,
 'Winter Wheat': 24,
 'Dbl Crop WinWht/Soybeans': 26,
 'Alfalfa': 36,
 'Other Hay/Non Alfalfa': 37,
 'Open Water': 111,
 'Developed/Open Space': 121,
 'Deciduous Forest': 141,
 'Grass/Pasture': 176,
 'Woody Wetlands': 190,
 'Herbaceous Wetlands': 195}

In [68]:
# Create a list of the crop types to be used for the headings of the error_matrix columns
crop_types = []
for type, label in cdl_name_class_combined.items():
    name = type
    crop_types.append(name)
crop_types

['Corn',
 'Soybeans',
 'Pop or Orn Corn',
 'Winter Wheat',
 'Dbl Crop WinWht/Soybeans',
 'Alfalfa',
 'Other Hay/Non Alfalfa',
 'Open Water',
 'Developed/Open Space',
 'Deciduous Forest',
 'Grass/Pasture',
 'Woody Wetlands',
 'Herbaceous Wetlands']

In [69]:
# Create output folder
if not os.path.exists("Outputs"):
    os.mkdir("Outputs")

In [70]:
# Train the Random Forest Classifier
model = (ee.Classifier.smileRandomForest(
        numberOfTrees = 80,
        variablesPerSplit = 3,
        minLeafPopulation = None,
        bagFraction = None,
        maxNodes = None,
        seed = None,
        )
    )

# Train the RF model with the training set
classifier = model.train(
        features = training_set,
        classProperty = "cropland",
        inputProperties = s2_monthly_median_masked.bandNames()
    )

# Perform the classification
classified = s2_monthly_median_masked.classify(classifier)


# Extract the corresponding values of the testing points from the classified image
validation = classified.sampleRegions(
        collection= testing_sample,
        properties= ["cropland"],
        scale=10
        )

""" 
validation = s2_median_2023.sampleRegions(
  collection = testing_sample,
  properties=  ["remapped"],
  scale=  10,
  )

test = validation.classify(classifier)
"""

# Get the error matrix and print its dimensions
error_matrix = validation.errorMatrix(actual="cropland", predicted="classification")
error_matrix_array = error_matrix.array().getInfo()

# Print the error matrix array
print("Error matrix array:", error_matrix_array)

# Check the dimensions of the error matrix
num_rows = len(error_matrix_array)
num_columns = len(error_matrix_array[0]) if num_rows > 0 else 0
print(f"Error matrix dimensions: {num_rows} rows x {num_columns} columns")

# Prepare data for DataFrame (excluding the first row and column)
data = [row[1:] for row in error_matrix_array[1:]]

# Define column names based on the first row (excluding the first element)
columns = crop_types

# Check if the number of columns matches
if len(columns) != len(data[0]):
    raise ValueError(f"Number of columns ({len(columns)}) does not match data ({len(data[0])})")

# Create DataFrame for the error matrix
error_matrix_df = pd.DataFrame(data, columns=columns)
print(error_matrix_df)

# Save the error matric to CSV
error_matrix_df.to_csv("Outputs/rf_error_matrix_crop_US.csv")


# Print accuracy metrics
print("Overall Accuracy for Random Forest:", round(error_matrix.accuracy().getInfo(), 3))
print("Producers Accuracy for Random Forest:", error_matrix.producersAccuracy().getInfo())
print("Consumers Accuracy for Random Forest:", error_matrix.consumersAccuracy().getInfo())
print("Kappa for Random Forest:", round(error_matrix.kappa().getInfo(), 3))


 # Get feature importance and values from the classifier
explained = classifier.explain()
importance = ee.Dictionary(explained.get("importance"))
feature_names = importance.keys().getInfo()
importance_values = importance.values().getInfo()
#importance_values = list(importance.values().getInfo().values())  # Convert values to list explicitly
    
# Combine the feature names and values into a DataFrame
importance_combined = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance_values
    })

# Normalize feature importance values to [0, 1]
scaler = MinMaxScaler()
    
# Flatten and reshape importance values before normalization
importance_values = [[v] for v in importance_combined['Importance'].values]
importance_values_norm = scaler.fit_transform(importance_values)

# Convert normalized values to percentages
importance_values_norm_percent = (importance_values_norm / importance_values_norm.sum()) * 100

# Convert original values to percentages
importance_values_percent = (importance_combined["Importance"] / importance_combined["Importance"].sum()) * 100

# Update importance_combined with percentage values
importance_combined['Importance_Norm'] = importance_values_norm.flatten()
importance_combined['Importance_Norm_Percent'] = importance_values_norm_percent.flatten()
importance_combined['Importance_Percent'] = importance_values_percent

importance_combined.sort_values(ascending=False, by="Importance").head(11)

# Save normalized feature importance to CSV
importance_combined.to_csv("Outputs/rf_feature_importance_crop_US.csv", index=False)

Error matrix array: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 12, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0], [0, 3, 14, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0], [0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 15, 4, 0, 2, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 3, 14, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 11, 6, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 10, 4, 0, 2, 0, 3, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 10, 0, 1, 0, 0, 1], [0, 2, 1, 1, 0, 0, 0, 3, 0, 6, 0, 2, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4, 4, 0], [0, 1, 0, 0, 0, 0, 0, 2, 0, 3, 1, 10, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 10, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 14]]
Error matrix dimensions: 14 rows x 14 columns
    Corn  Soybeans  Pop or Orn Corn  Winter Wheat  Dbl Crop WinWht/Soybeans  \
0     12         1                0             0                         0   
1      3        14                0             0                         0   
2      0         0               11             0                   

In [139]:
# Convert the binary image to integer with values 0 and 1
#classified_int = classified.multiply(255).toInt()

#### 6. RESULTS VISUALIZATION

In [71]:
# Visualize the result of the S-2 Crop Layer(10m) and the CDL (30m)
crop_vis_params = {"min": 1, "max":13, "palette" : ["#ffd300", "#267000", "#dda50a", "#a57000", "#707000", "#ffa5e2", "#a5f28c", "#4970a3", "#999999", "#93cc93", "#e8ffbf", "#7cafaf", "#7cafaf"]}

cdl_params = (
    cdl_crop,
    {},
    "Original Crop Layer (30m)"
)

rf_crop_type_params= (
    classified,
    crop_vis_params,
    "Classified Crop Type Map (10m)"
)

sliderMap(cdl_params, rf_crop_type_params)

Map(center=[40.7959, -84.4795], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [72]:
# Visualize the result of the S-2 Crop Layer(10m) and the Sentinel-2 Median Composite
s2_vis_params = {"min": 0, "max":0.3, "bands": ["B4_3", "B3_3", "B2_3"], "gamma":0.8}

s2_params = (
    s2_monthly_median_masked,
    s2_vis_params,
    "Sentinel-2 (September Median Composite)"
)

rf_crop_type_params= (
    classified,
    crop_vis_params,
    "Classified Crop Type Map (10m)"
)

sliderMap(s2_params, rf_crop_type_params)

Map(center=[40.7959, -84.4795], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [73]:
# Create a Linked Map to visualize the S-2 Crop Layer(10m), CDL (30m), Sentinel Monthly Median Composite, & NAIP (0.6m) 

# Define a list of layers
lc_layers = [classified,cdl_crop,  naip, s2_monthly_median_masked,] 

# Define the visualization parameters
viz_params = [
    crop_vis_params,
    {},
    viz_params_naip,
    s2_vis_params
]
 
    
# Create labels for the link map
labels = [
    "Classified Crop Type Map (10m)",
    "Original Crop Layer (30m)",
     "NAIP (0.6m)",
    "Sentinel-2 (Monthly Median Composite)"
]

# Visualize the layers
linkedMap(lc_layers, viz_params, labels)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

### 7. EXPORT RESULTS

In [74]:
# Export the study area feature

# Convert the Geometry to a FeatureCollection
roi_feature = ee.Feature(roi)
roi_feature_collection = ee.FeatureCollection([roi_feature])

roi_Export = ee.batch.Export.table.toDrive(
    collection = roi_feature_collection, 
    description= "US_Crop_Extent", 
    folder = "US_Crop", 
    fileFormat = "SHP",
)

roi_Export.start()
roi_Export.status()

{'state': 'READY',
 'description': 'US_Crop_Extent',
 'priority': 100,
 'creation_timestamp_ms': 1764244489659,
 'update_timestamp_ms': 1764244489659,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'X7DLFLU6BXUZ7MINLXORIASU',
 'name': 'projects/630402930502/operations/X7DLFLU6BXUZ7MINLXORIASU'}

In [75]:
roi_Export.status()

{'state': 'READY',
 'description': 'US_Crop_Extent',
 'priority': 100,
 'creation_timestamp_ms': 1764244489659,
 'update_timestamp_ms': 1764244489659,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'X7DLFLU6BXUZ7MINLXORIASU',
 'name': 'projects/630402930502/operations/X7DLFLU6BXUZ7MINLXORIASU'}

In [76]:
# Export the CDL (30m)
cdl_crop_Export = ee.batch.Export.image.toDrive(
    image = cdl_crop,
    description = "CDL_CROP_30m",
    folder = "US_Crop",
    region = roi,
    scale = 30,
    maxPixels = 1e13
)

cdl_crop_Export.start()
cdl_crop_Export.status()

{'state': 'READY',
 'description': 'CDL_CROP_30m',
 'priority': 100,
 'creation_timestamp_ms': 1764244499686,
 'update_timestamp_ms': 1764244499686,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'U75ZQPAWNQEFKDNCCQ23K46D',
 'name': 'projects/630402930502/operations/U75ZQPAWNQEFKDNCCQ23K46D'}

In [77]:
classified.bandTypes().getInfo()
cdl_crop.bandTypes().getInfo()

{'cropland': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [78]:
# Export the CDL masked (10m)
cdl_masked_Export = ee.batch.Export.image.toDrive(
    image = cdl_crop_class_masked,
    description = "CDL_CROP_MASKED_10m",
    folder = "US_Crop",
    region = roi,
    scale = grid_scale,
    maxPixels = 1e13
)

cdl_masked_Export.start()
cdl_masked_Export.status()

NameError: name 'cdl_crop_class_masked' is not defined

In [79]:
# Export the classification (10m)
classified_Export = ee.batch.Export.image.toDrive(
    image = classified,
    description = "S2_CLASSIFIED_10m",
    folder = "US_Crop",
    region = roi,
    scale = grid_scale,
    maxPixels = 1e13
)

classified_Export.start()
classified_Export.status()

{'state': 'READY',
 'description': 'S2_CLASSIFIED_10m',
 'priority': 100,
 'creation_timestamp_ms': 1764244544686,
 'update_timestamp_ms': 1764244544686,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'CALVBZOJVKK5W3JUIUEWHUFG',
 'name': 'projects/630402930502/operations/CALVBZOJVKK5W3JUIUEWHUFG'}

In [80]:
# Export the NAIP Image (0.6m)
naip_export = naip
naip_Export = ee.batch.Export.image.toDrive(
    image = naip_export,
    description = "NAIP_US_CROP_0_6m",
    folder = "US_Crop",
    region = roi,
    scale = 0.6,
    maxPixels = 1e13
)

naip_Export.start()
naip_Export.status()

{'state': 'READY',
 'description': 'NAIP_US_CROP_0_6m',
 'priority': 100,
 'creation_timestamp_ms': 1764244548339,
 'update_timestamp_ms': 1764244548339,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '7KMKMZLGRHSRPCUJSWTWF7BU',
 'name': 'projects/630402930502/operations/7KMKMZLGRHSRPCUJSWTWF7BU'}

In [81]:
naip_Export.status()

{'state': 'READY',
 'description': 'NAIP_US_CROP_0_6m',
 'priority': 100,
 'creation_timestamp_ms': 1764244548339,
 'update_timestamp_ms': 1764244548339,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '7KMKMZLGRHSRPCUJSWTWF7BU',
 'name': 'projects/630402930502/operations/7KMKMZLGRHSRPCUJSWTWF7BU'}

In [82]:
# Export the Sentinel Monthly Median Comoposite Image (10m)
s2_median_export = s2_monthly_median.select(['B2_5', 'B3_5', 'B4_5', 'B8_5', 'B5_5', 'B6_5', 'B7_5', 'B8A_5', 'B11_5', 'B12_5']).clip(roi)

s2_Export = ee.batch.Export.image.toDrive(
    image = s2_median_export,
    description = "S2_AUG_10m",
    folder = "US_Crop",
    region = roi,
    scale = grid_scale,
    maxPixels = 1e13
)

s2_Export.start()
s2_Export.status()

{'state': 'READY',
 'description': 'S2_AUG_10m',
 'priority': 100,
 'creation_timestamp_ms': 1764244553076,
 'update_timestamp_ms': 1764244553076,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '674ZSXVHBWCCPP3GUTKQG6JF',
 'name': 'projects/630402930502/operations/674ZSXVHBWCCPP3GUTKQG6JF'}

In [83]:
s2_Export.status()

{'state': 'READY',
 'description': 'S2_AUG_10m',
 'priority': 100,
 'creation_timestamp_ms': 1764244553076,
 'update_timestamp_ms': 1764244553076,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '674ZSXVHBWCCPP3GUTKQG6JF',
 'name': 'projects/630402930502/operations/674ZSXVHBWCCPP3GUTKQG6JF'}